In [ ]:
import fnmatch
import cv2
import numpy as np
import string
import time
from tensorflow.keras.utils import pad_sequences
from keras.layers import Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional
from keras.models import Model
from keras.activations import relu, sigmoid, softmax
import keras.backend as K
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import os
import tensorflow as tf
import random
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
char_list = string.ascii_letters+string.digits

In [ ]:
def encode_to_labels(txt):
   
    dig_lst = []
    for index, char in enumerate(txt):
        try:
            dig_lst.append(char_list.index(char))
        except:
            print(char)
        
    return dig_lst

def find_dominant_color(image):

        width, height = 150,150
        image = image.resize((width, height),resample = 0)
 
        pixels = image.getcolors(width * height)

        sorted_pixels = sorted(pixels, key=lambda t: t[0])

        dominant_color = sorted_pixels[-1][1]
        return dominant_color

def preprocess_img(img, imgSize):
    if img is None:
        img = np.zeros([imgSize[1], imgSize[0]]) 
        print("Image None!")

   
    (wt, ht) = imgSize
    (h, w) = img.shape
    fx = w / wt
    fy = h / ht
    f = max(fx, fy)
    newSize = (max(min(wt, int(w / f)), 1),
               max(min(ht, int(h / f)), 1))  
    img = cv2.resize(img, newSize, interpolation=cv2.INTER_CUBIC) 
                                                              
    most_freq_pixel=find_dominant_color(Image.fromarray(img))
    target = np.ones([ht, wt]) * most_freq_pixel  
    target[0:newSize[1], 0:newSize[0]] = img

    img = target

    return img

In [ ]:
training_img = []
training_txt = []
train_input_length = []
train_label_length = []
orig_txt = []
valid_img = []
valid_txt = []
valid_input_length = []
valid_label_length = []
valid_orig_txt = []
max_label_len = 0

annot=open('/content/drive/MyDrive/annotation.txt','r').readlines()
imagenames=[]
txts=[]

for cnt in annot:
    filename,txt=cnt.split(',')[0],cnt.split(',')[1].split('\n')[0]
    imagenames.append(filename)
    txts.append(txt)
    

    

    
for i in range(200):
        img = cv2.imread('/content/drive/MyDrive/images/'+imagenames[i],0)   
 
        img=preprocess_img(img,(128,32))
        img=np.expand_dims(img,axis=-1)
        img = img/255.
        txt = txts[i]
        

        if len(txt) > max_label_len:
            max_label_len = len(txt)
            
           

        if i%10 == 0:     
            valid_orig_txt.append(txt)   
            valid_label_length.append(len(txt))
            valid_input_length.append(31)
            valid_img.append(img)
            valid_txt.append(encode_to_labels(txt))
        else:
            orig_txt.append(txt)   
            train_label_length.append(len(txt))
            train_input_length.append(31)
            training_img.append(img)
            training_txt.append(encode_to_labels(txt)) 
        
        
        if i == 150000:
            flag = 1
            break
        i+=1

In [ ]:
train_padded_txt = pad_sequences(training_txt, maxlen=max_label_len, padding='post', value = len(char_list))
valid_padded_txt = pad_sequences(valid_txt, maxlen=max_label_len, padding='post', value = len(char_list))

In [ ]:
print(training_txt)

[[3, 17, 20, 0, 11, 15, 20], [4, 1], [22, 9, 7, 23, 16, 15, 24, 8], [23, 22, 24, 7, 18, 25, 17, 6, 6, 21, 13, 21, 21, 20, 15, 24, 22], [15, 11, 11, 23, 23, 18, 14, 8, 18, 10, 7, 12, 0, 2, 23], [11, 3, 7, 14], [12, 9, 4, 9, 8, 5, 13, 7, 9, 10, 0, 13, 4, 6, 20, 14, 10], [11, 25, 14, 0, 9], [2, 25, 2, 24, 5], [19, 19, 15, 20, 9, 4, 19, 17, 21], [17, 6, 15, 17, 15, 25, 2], [7, 16, 14, 12, 5, 21], [22, 5, 14, 11, 15, 7, 4, 5, 5, 21, 24], [4, 2, 0, 10, 1, 7, 12, 23, 19, 21, 3, 24, 22, 2, 0, 24, 0, 22], [14, 5, 22, 13, 13, 3, 6, 1, 5, 2], [17, 12, 10, 11, 19, 11, 23, 2, 19, 9, 20, 25, 2], [9, 16, 9, 21, 21, 14], [0, 7, 4, 17, 23, 6, 9, 21, 10, 22], [3, 25, 19, 22, 8, 17, 13, 13, 24], [6, 21, 22, 22, 14, 7, 9, 18, 0, 11, 25, 20], [3, 12, 8, 16, 0, 13, 5, 16, 9, 20, 22, 14, 18, 11, 22], [21, 24, 3, 3, 6, 16, 15, 15], [2, 0, 2, 4, 19, 24, 14, 14, 24, 11, 22, 3, 19, 14, 16, 17, 17], [24, 8, 16, 15, 12, 10, 19, 24, 17, 18, 7, 21], [13, 14, 17, 18, 22, 19, 24, 8, 21], [12, 2], [1, 8, 12, 24, 14, 15

In [ ]:
inputs = Input(shape=(32,128,1))
conv_1 = Conv2D(64, (3,3), activation = 'relu', padding='same')(inputs)
pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)
conv_2 = Conv2D(128, (3,3), activation = 'relu', padding='same')(pool_1)
pool_2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)
conv_3 = Conv2D(256, (3,3), activation = 'relu', padding='same')(pool_2)
conv_4 = Conv2D(256, (3,3), activation = 'relu', padding='same')(conv_3)
pool_4 = MaxPool2D(pool_size=(2, 1))(conv_4)
conv_5 = Conv2D(512, (3,3), activation = 'relu', padding='same')(pool_4)
batch_norm_5 = BatchNormalization()(conv_5)
conv_6 = Conv2D(512, (3,3), activation = 'relu', padding='same')(batch_norm_5)
batch_norm_6 = BatchNormalization()(conv_6)
pool_6 = MaxPool2D(pool_size=(2, 1))(batch_norm_6)
conv_7 = Conv2D(512, (2,2), activation = 'relu')(pool_6)
squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_7)
blstm_1 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(squeezed)
blstm_2 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(blstm_1)
outputs = Dense(len(char_list)+1, activation = 'softmax')(blstm_2)


act_model = Model(inputs, outputs)

 
def ctc_lambda_func(args):

    y_pred, labels, input_length, label_length = args
 
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

 




In [ ]:
act_model.load_weights('/content/drive/MyDrive/trainedmodes/CRNN_model.hdf5')
 


In [ ]:
img = cv2.imread('1.png',0)
img=preprocess_img(img,(128,32))
img=np.expand_dims(img,axis=-1)
t = []
t.append(img)
t = np.array(t)

In [ ]:
prediction = act_model.predict(t)
 

out = K.get_value(K.ctc_decode(prediction, input_length=np.ones(prediction.shape[0])*prediction.shape[1],
                         greedy=True)[0][0])
 


for x in out:
  
    print("predicted text = ", end = '')
    for p in x:  
        if int(p) != -1:
            print(char_list[int(p)], end = '')       
    print('\n')


1/1 [==============================] - 0s 59ms/step
predicted text = python



In [ ]:
cv2.imwrite('ma.png',valid_img[11])

True